In [27]:
import urllib
import requests
import datetime
from datetime import timedelta
from datetime import datetime
import json
import pickle

URL = 'https://st-dev-data-api.azurewebsites.net'
invertersURL = urllib.parse.urljoin(URL, '/api/v0.1/buildings/energyville1/pv/roof/inverters/')
powerURL = urllib.parse.urljoin(URL, '/api/v0.1/buildings/energyville1/pv/roof/power/')
user = 'eric_massip'
pswd = 'ssCpoDC3uHF4ssHJtvMf'

DATE_FORMAT_STR = "%Y-%m-%d"

start_time = datetime(2017, 1, 1)
end_time = datetime(2017, 1, 16)


def get_inverter_ids():
    inverter_ids_json_response = json.loads(requests.get(invertersURL, auth=(user, pswd)).text)['data']
    inverter_ids = []
    for i in range(len(inverter_ids_json_response)):
        inverter_ids.append(inverter_ids_json_response[i]['InverterId'])
    return inverter_ids


def get_params(inverter_id, start, end):
    return {'start': start.strftime(DATE_FORMAT_STR),
            'end': end.strftime(DATE_FORMAT_STR),
            'time_zone': 'Central European Standard Time',
            'inverter_ids': inverter_id}


def get_power_consumption(inverter_id, start, end):
    return json.loads(requests.get(powerURL, params=get_params(inverter_id, start, end), auth=(user, pswd)).text)['data']


def get_power_consumptions(start, end):
    return [get_power_consumption(inverter_id, start, end) for inverter_id in get_inverter_ids()]


power_consumptions = get_power_consumptions(start_time, end_time)
#pickle.dump(power_consumptions, open('power_consumptions', 'wb'))


In [28]:
#start_time = datetime(2017, 1, 1)
#end_time = datetime(2017, 1, 2)


def get_time_dictionary_every_5min(start_time, end_time):
    timedelta_5 = timedelta(minutes=5)
    
    dates = []
    date = start_time
    while date != end_time:
        dates.append(date)
        date = date + timedelta_5
        
    return dates


dates = get_time_dictionary_every_5min(start_time, end_time)
#pickle.dump(dates, open('dates', 'wb'))

In [29]:
#power_consumptions = pickle.load(open('power_consumptions_6_1_2017', mode='rb'))
#dates = pickle.load(open('dates_6_1_2017', mode='rb'))


def get_total_power_consumption():
    total_power_consumption = []
    for date_to_be_checked in dates:
        date_to_be_checked_str = date_to_be_checked.strftime("%Y-%m-%d %H:%M:%S")
        power_sum = 0
        for inverter_consumption in power_consumptions:
            for i in range(len(inverter_consumption)):
                this_inverter_consumption = inverter_consumption[i]
                log_generation_power = this_inverter_consumption['GenerationPower']
                if log_generation_power != 0:
                    log_datetime = datetime.fromisoformat(this_inverter_consumption['DateTimeMeasurement']).strftime("%Y-%m-%d %H:%M:%S")
                    if log_datetime == date_to_be_checked_str:
                        if log_generation_power != 0:
                            power_sum += log_generation_power
        #print(date_to_be_checked_str)
        total_power_consumption.append({'date': date_to_be_checked_str, 'power_sum': power_sum})
    return total_power_consumption


total_power_consumption = get_total_power_consumption()


KeyboardInterrupt: 

In [ ]:
import pandas as pd

df = pd.DataFrame(total_power_consumption)

df.to_csv('total_power_consumption_1_1_2017')


In [31]:
import pickle

total_power_consumption_1_1_2017 = pickle.load(open('pv/total_power_consumption_1_1_2017', mode='rb'))

In [33]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(total_power_consumption_1_1_2017)

[datetime.datetime(2017, 1, 1, 0, 0), datetime.datetime(2017, 1, 1, 0, 5), datetime.datetime(2017, 1, 1, 0, 10), datetime.datetime(2017, 1, 1, 0, 15), datetime.datetime(2017, 1, 1, 0, 20), datetime.datetime(2017, 1, 1, 0, 25), datetime.datetime(2017, 1, 1, 0, 30), datetime.datetime(2017, 1, 1, 0, 35), datetime.datetime(2017, 1, 1, 0, 40), datetime.datetime(2017, 1, 1, 0, 45), datetime.datetime(2017, 1, 1, 0, 50), datetime.datetime(2017, 1, 1, 0, 55), datetime.datetime(2017, 1, 1, 1, 0), datetime.datetime(2017, 1, 1, 1, 5), datetime.datetime(2017, 1, 1, 1, 10), datetime.datetime(2017, 1, 1, 1, 15), datetime.datetime(2017, 1, 1, 1, 20), datetime.datetime(2017, 1, 1, 1, 25), datetime.datetime(2017, 1, 1, 1, 30), datetime.datetime(2017, 1, 1, 1, 35), datetime.datetime(2017, 1, 1, 1, 40), datetime.datetime(2017, 1, 1, 1, 45), datetime.datetime(2017, 1, 1, 1, 50), datetime.datetime(2017, 1, 1, 1, 55), datetime.datetime(2017, 1, 1, 2, 0), datetime.datetime(2017, 1, 1, 2, 5), datetime.datetime

In [82]:
import pprint
#print(total_power_consumption)

for consumption in total_power_consumption:
    #print(consumption)
    hour = datetime.fromisoformat(consumption['date']).hour
    minute = datetime.fromisoformat(consumption['date']).minute
    if consumption['power_sum'] > 0 and hour == 10 and minute == 0:
        pprint.pprint(consumption)

{'date': '2017-01-01 10:00:00', 'power_sum': 12802.0}
{'date': '2017-01-02 10:00:00', 'power_sum': 422.0}


In [1]:
import pandas as pd

total_power_consumption_1_1_2017 = pd.read_csv('tmp/total_power_consumption_1_1_2017.csv')
total_power_consumption_1_2_2017 = pd.read_csv('tmp/total_power_consumption_1_2_2017.csv')
total_power_consumption_2_1_2017 = pd.read_csv('tmp/total_power_consumption_2_1_2017.csv')
total_power_consumption_2_2_2017 = pd.read_csv('tmp/total_power_consumption_2_2_2017.csv')
total_power_consumption_3_1_2017 = pd.read_csv('tmp/total_power_consumption_3_1_2017.csv')
total_power_consumption_3_2_2017 = pd.read_csv('tmp/total_power_consumption_3_2_2017.csv')
total_power_consumption_4_1_2017 = pd.read_csv('tmp/total_power_consumption_4_1_2017.csv')
total_power_consumption_4_2_2017 = pd.read_csv('tmp/total_power_consumption_4_2_2017.csv')
total_power_consumption_5_1_2017 = pd.read_csv('tmp/total_power_consumption_5_1_2017.csv')
total_power_consumption_5_2_2017 = pd.read_csv('tmp/total_power_consumption_5_2_2017.csv')
total_power_consumption_6_1_2017 = pd.read_csv('tmp/total_power_consumption_6_1_2017.csv')
total_power_consumption_6_2_2017 = pd.read_csv('tmp/total_power_consumption_6_2_2017.csv')
total_power_consumption_7_1_2017 = pd.read_csv('tmp/total_power_consumption_7_1_2017.csv')
total_power_consumption_7_2_2017 = pd.read_csv('tmp/total_power_consumption_7_2_2017.csv')
total_power_consumption_8_1_2017 = pd.read_csv('tmp/total_power_consumption_8_1_2017.csv')
total_power_consumption_8_2_2017 = pd.read_csv('tmp/total_power_consumption_8_2_2017.csv')
total_power_consumption_9_1_2017 = pd.read_csv('tmp/total_power_consumption_9_1_2017.csv')
total_power_consumption_9_2_2017 = pd.read_csv('tmp/total_power_consumption_9_2_2017.csv')
total_power_consumption_10_1_2017 = pd.read_csv('tmp/total_power_consumption_10_1_2017.csv')
total_power_consumption_10_2_2017 = pd.read_csv('tmp/total_power_consumption_10_2_2017.csv')
total_power_consumption_11_1_2017 = pd.read_csv('tmp/total_power_consumption_11_1_2017.csv')
total_power_consumption_11_2_2017 = pd.read_csv('tmp/total_power_consumption_11_2_2017.csv')
total_power_consumption_12_1_2017 = pd.read_csv('tmp/total_power_consumption_12_1_2017.csv')
total_power_consumption_12_2_2017 = pd.read_csv('tmp/total_power_consumption_12_2_2017.csv')

In [2]:
total_power_consumption_1_1_2017 = total_power_consumption_1_1_2017[['date', 'power_sum']]
total_power_consumption_1_2_2017 = total_power_consumption_1_2_2017[['date', 'power_sum']]
total_power_consumption_2_1_2017 = total_power_consumption_2_1_2017[['date', 'power_sum']]
total_power_consumption_2_2_2017 = total_power_consumption_2_2_2017[['date', 'power_sum']]
total_power_consumption_3_1_2017 = total_power_consumption_3_1_2017[['date', 'power_sum']]
total_power_consumption_3_2_2017 = total_power_consumption_3_2_2017[['date', 'power_sum']]
total_power_consumption_4_1_2017 = total_power_consumption_4_1_2017[['date', 'power_sum']]
total_power_consumption_4_2_2017 = total_power_consumption_4_2_2017[['date', 'power_sum']]
total_power_consumption_5_1_2017 = total_power_consumption_5_1_2017[['date', 'power_sum']]
total_power_consumption_5_2_2017 = total_power_consumption_5_2_2017[['date', 'power_sum']]
total_power_consumption_6_1_2017 = total_power_consumption_6_1_2017[['date', 'power_sum']]
total_power_consumption_6_2_2017 = total_power_consumption_6_2_2017[['date', 'power_sum']]
total_power_consumption_7_1_2017 = total_power_consumption_7_1_2017[['date', 'power_sum']]
total_power_consumption_7_2_2017 = total_power_consumption_7_2_2017[['date', 'power_sum']]
total_power_consumption_8_1_2017 = total_power_consumption_8_1_2017[['date', 'power_sum']]
total_power_consumption_8_2_2017 = total_power_consumption_8_2_2017[['date', 'power_sum']]
total_power_consumption_9_1_2017 = total_power_consumption_9_1_2017[['date', 'power_sum']]
total_power_consumption_9_2_2017 = total_power_consumption_9_2_2017[['date', 'power_sum']]
total_power_consumption_10_1_2017 = total_power_consumption_10_1_2017[['date', 'power_sum']]
total_power_consumption_10_2_2017 = total_power_consumption_10_2_2017[['date', 'power_sum']]
total_power_consumption_11_1_2017 = total_power_consumption_11_1_2017[['date', 'power_sum']]
total_power_consumption_11_2_2017 = total_power_consumption_11_2_2017[['date', 'power_sum']]
total_power_consumption_12_1_2017 = total_power_consumption_12_1_2017[['date', 'power_sum']]
total_power_consumption_12_2_2017 = total_power_consumption_12_2_2017[['date', 'power_sum']]

In [3]:
total_power_consumption_2017 = pd.concat([total_power_consumption_1_1_2017,
total_power_consumption_1_2_2017,
total_power_consumption_2_1_2017,
total_power_consumption_2_2_2017,
total_power_consumption_3_1_2017,
total_power_consumption_3_2_2017,
total_power_consumption_4_1_2017,
total_power_consumption_4_2_2017,
total_power_consumption_5_1_2017,
total_power_consumption_5_2_2017,
total_power_consumption_6_1_2017,
total_power_consumption_6_2_2017,
total_power_consumption_7_1_2017,
total_power_consumption_7_2_2017,
total_power_consumption_8_1_2017,
total_power_consumption_8_2_2017,
total_power_consumption_9_1_2017,
total_power_consumption_9_2_2017,
total_power_consumption_10_1_2017,
total_power_consumption_10_2_2017,
total_power_consumption_11_1_2017,
total_power_consumption_11_2_2017,
total_power_consumption_12_1_2017,
total_power_consumption_12_2_2017], ignore_index=True)

In [4]:
total_power_consumption_2017.to_csv('total_power_consumption_2017.csv')

In [13]:
day_1 = pv.head(288)

In [16]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(day_1)

     Unnamed: 0                 date   power_sum
0             0  2017-01-01 00:00:00         0.0
1             1  2017-01-01 00:05:00         0.0
2             2  2017-01-01 00:10:00         0.0
3             3  2017-01-01 00:15:00         0.0
4             4  2017-01-01 00:20:00         0.0
5             5  2017-01-01 00:25:00         0.0
6             6  2017-01-01 00:30:00         0.0
7             7  2017-01-01 00:35:00         0.0
8             8  2017-01-01 00:40:00         0.0
9             9  2017-01-01 00:45:00         0.0
10           10  2017-01-01 00:50:00         0.0
11           11  2017-01-01 00:55:00         0.0
12           12  2017-01-01 01:00:00         0.0
13           13  2017-01-01 01:05:00         0.0
14           14  2017-01-01 01:10:00         0.0
15           15  2017-01-01 01:15:00         0.0
16           16  2017-01-01 01:20:00         0.0
17           17  2017-01-01 01:25:00         0.0
18           18  2017-01-01 01:30:00         0.0
19           19  201